# AI Agents Crash Course - Part 2 - Structured Output

## Project Setup

This notebook will guide you through setting up the project environment for using CrewAI. We will:

1. Install the required Python modules.
2. Set up a virtual environment.
3. Verify the installation.

In [ ]:
# Uncomment if you are not using devcontainers and want to set up a local environment
# 
# # Step 1: Create and activate a virtual environment
# #
# %python3 -m venv venv
# %source venv/bin/activate
# 
# # Step 2: Install required Python modules
# #
# %pip install -r requirements.txt
# 
# # Step 3: Verify installation
# #
# %pip list

## Load Required Python Modules and Libraries

In [1]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown
from crewai import LLM, Agent, Task, Crew

## [Optional] Enable litellm debug logging

In [ ]:
# Uncomment in order to enable litellm debugging for better error diagnostics
# NOTE: You will have to restart the jupyter kernel to disable debug logging once it has been enabled.
#
# import litellm
# litellm._turn_on_debug()

## Load Environment Variables and Configure LLM

This block loads environment variables from the `.env` file, including the OpenAI API Key, which is required to authenticate with OpenAI's services. It then configures the `LLM` object to use OpenAI's GPT-4 model. Alternatively, you can uncomment the provided code to configure the `LLM` object to use Ollama with a local model, provided Ollama is installed and running.

In [ ]:
# Load environment variables from .env file
# Note: In devcontainer, variables are already loaded by dotenv feature,
# but load_dotenv() is safe and won't override existing environment variables
load_dotenv()

# Uncomment the code block below to use OpenAI with your API Key
# 
# api_key = os.getenv('OPENAI_API_KEY')
# if not api_key:
#     raise ValueError("OPENAI_API_KEY is not set in the .env file")

# Uncomment the code block below to use ollama
# 
OLLAMA_API_BASE = os.getenv('OLLAMA_API_BASE')
if not OLLAMA_API_BASE:
    raise ValueError("OLLAMA_API_BASE is not set in the .env file")

## Configure LLM

Configures the `LLM` object to use OpenAI's GPT-4 model. 

Alternatively, you can uncomment the provided code to configure the `LLM` object to use Ollama with a local model, provided Ollama is installed and running.

In [ ]:
# llm = LLM(
#     model="gpt-4o",  # Specify the OpenAI model you want to use
#     api_key=api_key
# )

# Uncomment the code block below to use Ollama with your local model
# Make sure to have Ollama installed and running
# 
llm = LLM(
    # model="ollama/llama3:latest",
    # model="ollama/llama3.2:1b",
    # model="ollama/deepseek-r1:latest",
    # model="ollama/gemma3:latest",
    model="ollama/gemma3n:latest",
    base_url=OLLAMA_API_BASE
)

## Verify LLM Configuration

### Universal LLM Connection Test

In [ ]:
import requests
import os

def test_llm_connection():
    """Test LLM connection regardless of provider"""
    
    print("=== LLM Configuration Analysis ===")
    
    # Analyze LLM configuration
    model_name = getattr(llm, 'model', 'Unknown')
    base_url = getattr(llm, 'base_url', None)
    api_key_set = bool(getattr(llm, 'api_key', None))
    
    print(f"Model: {model_name}")
    print(f"Base URL: {base_url if base_url else 'Default (provider-specific)'}")
    print(f"API Key Set: {'Yes' if api_key_set else 'No'}")
    
    # Determine provider type
    provider = "unknown"
    if "ollama" in model_name.lower():
        provider = "ollama"
    elif "gpt" in model_name.lower() or "openai" in model_name.lower():
        provider = "openai"
    elif "claude" in model_name.lower() or "anthropic" in model_name.lower():
        provider = "anthropic"
    elif "gemini" in model_name.lower() or "google" in model_name.lower():
        provider = "google"
    
    print(f"Detected Provider: {provider}")
    
    # Provider-specific connection tests
    print(f"\n=== {provider.title()} Connection Test ===")
    
    if provider == "ollama" and base_url:
        try:
            # Test Ollama server availability
            test_url = f"{base_url}/api/tags"
            response = requests.get(test_url, timeout=5)
            print(f"Ollama Server Status: {response.status_code}")
            if response.status_code == 200:
                models = response.json().get('models', [])
                print(f"Available Models: {len(models)} found")
                # Check if our specific model is available
                model_available = any(model_name.replace('ollama/', '') in str(model) for model in models)
                print(f"Target Model Available: {'Yes' if model_available else 'No'}")
            else:
                print(f"Ollama server responded with status: {response.status_code}")
        except Exception as e:
            print(f"❌ Ollama server connection failed: {e}")
    
    elif provider == "openai":
        print("OpenAI connection test (API key validation happens during LLM call)")
        api_key_env = os.getenv('OPENAI_API_KEY')
        print(f"OPENAI_API_KEY environment variable: {'Set' if api_key_env else 'Not set'}")
    
    elif provider == "anthropic":
        print("Anthropic connection test")
        api_key_env = os.getenv('ANTHROPIC_API_KEY')
        print(f"ANTHROPIC_API_KEY environment variable: {'Set' if api_key_env else 'Not set'}")
    
    elif provider == "google":
        print("Google AI connection test")
        api_key_env = os.getenv('GOOGLE_API_KEY')
        print(f"GOOGLE_API_KEY environment variable: {'Set' if api_key_env else 'Not set'}")
    
    else:
        print("Generic provider - will test with LLM call only")
    
    # Universal LLM functionality test
    print(f"\n=== LLM Functionality Test ===")
    try:
        test_response = llm.call([{"role": "user", "content": "Respond with exactly: 'Test successful'"}])
        print("✅ LLM call successful!")
        print(f"Response type: {type(test_response)}")
        
        # Try to extract response content
        if hasattr(test_response, 'content'):
            print(f"Response content: {test_response.content[:100]}...")
        elif isinstance(test_response, str):
            print(f"Response: {test_response[:100]}...")
        else:
            print(f"Response: {str(test_response)[:100]}...")
            
    except Exception as e:
        print(f"❌ LLM call failed: {e}")
        print(f"Error type: {type(e).__name__}")
        
        # Provide specific troubleshooting tips based on error
        error_str = str(e).lower()
        if "connection" in error_str or "timeout" in error_str:
            print("💡 Tip: Check network connection and base_url configuration")
        elif "api_key" in error_str or "authentication" in error_str or "unauthorized" in error_str:
            print("💡 Tip: Check API key configuration and permissions")
        elif "model" in error_str or "not found" in error_str:
            print("💡 Tip: Verify model name and availability")

# Run the comprehensive test
test_llm_connection()

### Environment Variables Check for All LLM Providers

In [ ]:
def check_environment_variables():
    """Check environment variables for all major LLM providers"""
    
    print("=== Environment Variables Status ===")
    
    # Common LLM provider environment variables
    env_vars = {
        "OpenAI": ["OPENAI_API_KEY", "OPENAI_BASE_URL"],
        "Anthropic": ["ANTHROPIC_API_KEY"],
        "Google": ["GOOGLE_API_KEY", "GOOGLE_APPLICATION_CREDENTIALS"],
        "Cohere": ["COHERE_API_KEY"],
        "Hugging Face": ["HUGGINGFACE_API_KEY", "HF_TOKEN"],
        "Ollama": ["OLLAMA_API_BASE", "OLLAMA_HOST"],
        "Azure OpenAI": ["AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT"],
        "AWS Bedrock": ["AWS_ACCESS_KEY_ID", "AWS_SECRET_ACCESS_KEY", "AWS_REGION"],
        "Together AI": ["TOGETHER_API_KEY"],
        "Replicate": ["REPLICATE_API_TOKEN"],
        "Perplexity": ["PERPLEXITYAI_API_KEY"],
        "Groq": ["GROQ_API_KEY"]
    }
    
    found_providers = []
    
    for provider, vars_list in env_vars.items():
        provider_vars = {}
        has_any_var = False
        
        for var in vars_list:
            value = os.getenv(var)
            if value:
                provider_vars[var] = "✅ Set"
                has_any_var = True
            else:
                provider_vars[var] = "❌ Not set"
        
        if has_any_var:
            found_providers.append(provider)
            print(f"\n{provider}:")
            for var, status in provider_vars.items():
                print(f"  {var}: {status}")
    
    if not found_providers:
        print("No LLM provider environment variables found.")
        print("Make sure to set the appropriate API keys for your chosen provider.")
    else:
        print(f"\nConfigured providers: {', '.join(found_providers)}")

# Run environment check
check_environment_variables()

## Building Crew Agents